<a href="https://colab.research.google.com/github/AdamChoong0095/ADS2002-33154384/blob/main/Catheter_project_tensor_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Catheater project (Neural Networks component)

Contributors:
Adam Choong
Louise Childs
Isaac Woods
Fengzhe Yang

The goal of this project is to determine whether CVC or NGC or ETT catheaters are "normally" placed or not. Hence, the approach taken will be a binary classificiation approach. We will be comparing neural network model configurations based on various metrics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import cv2
import os
import ast
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
RS=42#set random seed consistent do not changeb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train=pd.read_csv('/content/drive/MyDrive/ADS2002-Catheter-Team1/ranzcr-clip-catheter-line-classification/train.csv')
train_annots=pd.read_csv('/content/drive/MyDrive/ADS2002-Catheter-Team1/ranzcr-clip-catheter-line-classification/train_annotations.csv')

Since the entire dataset is too large and we need to be mindful of the RAM capacity, there needs to be a function in place to partially read a random sample of the dataset. We also need to organise the dataframes in such as way that it contains the images as the "feature variable" and the outcomes as the labels

## Handling the csv files and directories
A handful of useful functions will be defined below. Their main purposes will be:
* Cleaning the dataframe so that it includes only the instances with one outcome and possibly one annotation for each catheter type
* Forming the appropriate directory structures for ResNet and DenseNet and other models
* Setting file labels on dataframes
* Reading images

In [ ]:
def set_files_on_dataframe(df,tag='StudyInstanceUID'):
  '''
  df: target data frame
  tag: StudyInstanceUID
  '''
  if 'image_file' not in df.columns:
    df['image_file']=''
  for i,row in df.iterrows():
    if '.jpg' not in row[tag]:
      path=row[tag]+'.jpg'
      df._set_value(i,'image_file',path)
    elif '.jpg' in row[tag]:
      path=row[tag]
      df._set_value(i,'image_file',path)
  return df

In [ ]:
import shutil
#creates directories for training,validating and testing
def copy_files(destination_directory,source_directory,image_files):
  '''
  image files argument is a list of image files to move
  '''
  # Iterate through the image files and move them to the destination directory
  for image_file in image_files:
      source_path = os.path.join(source_directory,image_file)
      destination_path = os.path.join(destination_directory,image_file)
      # Check if the destination file already exists, and if it does, rename it to avoid overwriting
      if os.path.exists(destination_path):
          base, extension = os.path.splitext(image_file)
          count = 1
          while os.path.exists(destination_path):
              new_filename = f"{base}_{count}{extension}"
              destination_path = os.path.join(destination_directory, new_filename)
              count += 1
      # Move the image file to the destination directory
      if os.path.exists(destination_path):
        print(f"File '{image_file}' already exists in the destination directory. Skipping...")
      else:
        # Move the image file to the destination directory
        shutil.copy(source_path, destination_path)



In [ ]:
def organise_dir(origin,train_dir,val_dir,test_dir,cath_type,sample):
  '''
  args
  -origin: source file
  -train/test/val dir: location of folders containing files for testing training, etc.
  -cath_typle: catheter type we test on
  -sample: sample data frame we test on, must only have one type of catheter considered
  '''
  train_validate,test = train_test_split(sample, test_size=0.1,random_state=RS)
  train,validate = train_test_split(train_validate, test_size=0.11,random_state=RS)
  '''
  splits data too
  must match certain directory structure to work
  dataframe must match certain structure to work see above
  '''
  train_normal=train[train[cath_type+' - Normal'] == 1]
  train_abnormal=train[train[cath_type+' - Abnormal'] == 1]
  train_borderline=train[train[cath_type+' - Borderline'] == 1]
  validate_normal=validate[validate[cath_type+' - Normal'] == 1]
  validate_abnormal=validate[validate[cath_type+' - Abnormal'] == 1]
  validate_borderline=validate[validate[cath_type+' - Borderline'] == 1]
  test_normal=test[test[cath_type+' - Normal'] == 1]
  test_abnormal=test[test[cath_type+' - Abnormal'] == 1]
  test_borderline=test[test[cath_type+' - Borderline'] == 1]
  train_norm_dir=train_dir+'/Normal'
  train_border_dir=train_dir+'/Borderline'
  train_ab_dir=train_dir+'/Abnormal'
  val_norm_dir=val_dir+'/Normal'
  val_border_dir=val_dir+'/Borderline'
  val_ab_dir=val_dir+'/Abnormal'
  test_norm_dir=test_dir+'/Normal'
  test_border_dir=test_dir+'/Borderline'
  test_ab_dir=test_dir+'/Abnormal'
  train_list_normal=train_normal['image_file'].tolist()
  train_list_abnormal=train_abnormal['image_file'].tolist()
  train_list_borderline=train_borderline['image_file'].tolist()
  val_list_normal=validate_normal['image_file'].tolist()
  val_list_abnormal=validate_abnormal['image_file'].tolist()
  val_list_borderline=validate_borderline['image_file'].tolist()
  test_list_normal=test_normal['image_file'].tolist()
  test_list_abnormal=test_abnormal['image_file'].tolist()
  test_list_borderline=test_borderline['image_file'].tolist()
  copy_files(destination_directory=test_border_dir,source_directory=origin,image_files=test_list_borderline)
  copy_files(destination_directory=test_norm_dir,source_directory=origin,image_files=test_list_normal)
  copy_files(destination_directory=test_ab_dir,source_directory=origin,image_files=test_list_abnormal)
  copy_files(destination_directory=train_border_dir,source_directory=origin,image_files=train_list_borderline)
  copy_files(destination_directory=train_norm_dir,source_directory=origin,image_files=train_list_normal)
  copy_files(destination_directory=train_ab_dir,source_directory=origin,image_files=train_list_abnormal)
  copy_files(destination_directory=val_border_dir,source_directory=origin,image_files=val_list_borderline)
  copy_files(destination_directory=val_norm_dir,source_directory=origin,image_files=val_list_normal)
  copy_files(destination_directory=val_ab_dir,source_directory=origin,image_files=val_list_abnormal)


In [ ]:
def clean_dataframe(df,cath_type):
    '''
    only runs for train, not train annotations
    '''
    if cath_type == 'CVC' or cath_type == 'ETT':
        dataframe=df[['StudyInstanceUID',cath_type+' - Abnormal',cath_type+' - Normal',cath_type+' - Borderline','image_file']]
        dataframe=dataframe.drop(dataframe[(dataframe[cath_type+' - Abnormal'] == 0) & (dataframe[cath_type+' - Normal'] == 0) & (dataframe[cath_type+' - Borderline'] == 0)].index)
        dataframe=dataframe.drop(dataframe[(dataframe[cath_type+' - Abnormal'] == 1) & (dataframe[cath_type+' - Normal'] == 1)].index)
        dataframe=dataframe.drop(dataframe[(dataframe[cath_type+' - Borderline'] == 1) & (dataframe[cath_type+' - Normal'] == 1)].index)
        dataframe=dataframe.drop(dataframe[(dataframe[cath_type+' - Abnormal'] == 1) & (dataframe[cath_type+' - Borderline'] == 1)].index)
    elif cath_type == 'NGT':
        dataframe=df[['StudyInstanceUID','NGT - Abnormal','NGT - Normal','NGT - Borderline','NGT - Incompletely Imaged','image_file']]
        dataframe=dataframe.drop(dataframe[(dataframe['NGT - Abnormal'] == 0) & (dataframe['NGT - Normal'] == 0) & (dataframe['NGT - Borderline'] == 0) & (dataframe['NGT - Incompletely Imaged']==0)].index)
        dataframe=dataframe.drop(dataframe[(dataframe['NGT - Incompletely Imaged']==1)].index)
        dataframe=dataframe.drop(dataframe[(dataframe['NGT - Abnormal'] == 1) & (dataframe['NGT - Normal'] == 1)].index)
        dataframe=dataframe.drop(dataframe[(dataframe['NGT - Borderline'] == 1) & (dataframe['NGT - Normal'] == 1)].index)
        dataframe=dataframe.drop(dataframe[(dataframe['NGT - Abnormal'] == 1) & (dataframe['NGT - Borderline'] == 1)].index)
    return dataframe

In [ ]:
def image_reader(directory,file_name):
  path=directory+'/'+file_name
  img=cv2.imread(path)
  img=cv2.resize(img,dsize=(32,32))
  return img

In [ ]:
def set_image_on_df(df,dir):
    '''
    works as long as there is a StudyInstanceUID column
    '''
    df['image']=''
    for i in range(0,len(df)):
        image=image_reader(dir,df.iloc[i]['image_file'])
        image=image.astype('float')
        df.at[i, 'image'] = image
    return df

In [ ]:
set_files_on_dataframe(train)
ett=clean_dataframe(train,'ETT')
cvc=clean_dataframe(train,'CVC')
ngt=clean_dataframe(train,'NGT')

## Reading the data
We need to manipulate our dataset so it fulfils the mentioned criteria
* Multiclassification
* Relevant images only considered
* Feature extraction (highlighting annotations to increase pixel intensity)
* Well balanced variations

We will be using all images first but extending scalability will involve using only annotations


In [ ]:
train_annots=set_files_on_dataframe(train_annots)

In [ ]:
#source: https://www.kaggle.com/code/ihelon/catheter-position-exploratory-data-analysis?fbclid=IwAR1rhwS0ZMuVUkkUQMeP6TumSQFzpHMhypeCjb0IuVJcHCWVHtYVaKpwOjA
def plot_image_with_annotations(row_ind,df_annot,BASE_DIR):
    row = df_annot.iloc[row_ind]
    image_path = BASE_DIR+ "/"+row["StudyInstanceUID"] + ".jpg"
    label = row["label"]
    data = np.array(ast.literal_eval(row["data"]))
    plt.figure(figsize=(10, 5))
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.scatter(data[:, 0], data[:, 1])
    plt.suptitle(label, fontsize=15)

def visualize_annotations(file_id,BASE_DIR,df_annot):
    plt.figure(figsize=(8, 8))
    image = cv2.imread(BASE_DIR+ "/"+file_id+'.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)

    df_patient = df_annot.loc[df_annot["StudyInstanceUID"] == file_id]

    if df_patient.shape[0]:
        labels = df_patient["label"].values.tolist()
        lines = df_patient["data"].apply(ast.literal_eval).values.tolist()

        for line, label in zip(lines, labels):
            line = np.asarray(line)
            plt.scatter(line[:, 0], line[:, 1], s=40, label=label)

        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0, prop={'size': 20})

    plt.tick_params(axis="x", labelsize=15)
    plt.tick_params(axis="y", labelsize=15)
    plt.show()

def save_annot_plot(dest,row_ind,df_annot,source):
  row = df_annot.iloc[row_ind]
  plot_image_with_annotations(row_ind,df_annot,source)
  dest_path=dest+'/'+row["StudyInstanceUID"] + ".jpg"
  if os.path.exists(dest_path):
    pass
  else:
    plt.savefig(dest_path)


## Balancing datasets
There are too many normal images (see EDA brief overview). So we need to make sure there is a balance of the types of catheter placements to prevent meaningless results from being produced. Sampling using SMOTE or ADASYN requires tabular data in our dataframes for the `X` or feature space but these values are all array representations of image data. An easy way to fix this may be to use only `y` and consider `X` as a dummy column. Let's try both methods to see which is better for our problem.

In [ ]:
from sklearn.preprocessing import LabelEncoder
def encoder(outcomes,cath_type):
  '''
  works only with train file
  '''
  outcomes['Outcomes']=''
  for i in range(len(outcomes)):
    if outcomes.loc[i,cath_type+' - Normal']==1:
      outcomes.loc[i,'Outcomes']=cath_type+' - Normal'
    elif outcomes.loc[i,cath_type+' - Abnormal']==1:
      outcomes.loc[i,'Outcomes']=cath_type+' - Abnormal'
    elif outcomes.loc[i,cath_type+' - Borderline']==1:
      outcomes.loc[i,'Outcomes']=cath_type+' - Borderline'
  outcomes=outcomes.drop([cath_type+' - Normal',cath_type+' - Abnormal',cath_type+' - Borderline'],axis=1)
  le=LabelEncoder()
  outcomes['Outcomes']=le.fit_transform(outcomes['Outcomes'])
  return outcomes


In [ ]:
from imblearn.over_sampling import SMOTE,ADASYN
directory='/content/drive/MyDrive/ADS2002-Catheter-Team1/image_collection'
mask = train['image_file'].isin(os.listdir(directory))
sample_df = train[mask]
sample_df=clean_dataframe(sample_df,'CVC')
sample_df.reset_index(inplace=True)
sample_df.drop('index',axis=1,inplace=True)
encoded=encoder(sample_df,'CVC')

TypeError: ignored

In [ ]:
mask = train['image_file'].isin(os.listdir(directory))
sample_df = train[mask]
sample_df=clean_dataframe(sample_df,'CVC')
sample_df.reset_index(inplace=True)
sample_df.drop('index',axis=1,inplace=True)
sample_df=set_image_on_df(sample_df,directory)
outcomes=sample_df[["CVC - Abnormal",'CVC - Normal','CVC - Borderline']]


X_train, X_test, y_train, y_test = train_test_split(image_data,outcomes, test_size=0.2, random_state=RS)
X_train = np.array([np.ravel(image) for image in X_train])
X_test = np.array([np.ravel(image) for image in X_test])
X_train = X_train / 255.0
X_test = X_test / 255.0
y_train = np.array(y_train)
y_test = np.array(y_test)

## Moving images to folders
The directories need to be separated by training, testing and validating sets with each folder having preclassified images in each of them

In [ ]:
#check if moved properly
def count_files_in_directory(directory):
    file_count = 0

    # Walk through the directory and its subdirectories
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_count += 1
    return file_count

## Convolutional Neural Network construction
The purpose of this section will be to use ResNet and other models for transfer learning to compare their performances

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 1])
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)

In [ ]:
#supervised learning using custom CNN that uses softmax for multiclassification
custom = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

custom.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
hist=custom.fit(X_train, y_train, epochs=20, batch_size=4, validation_split=0.2)

# Evaluate your model
test_loss, test_accuracy = custom.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
plot_loss(hist)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    'images_split/train',
    target_size=(224, 224),  # ResNet-50 input size
    batch_size=32,
    class_mode='categorical'  # Use 'categorical' for multi-class classification
)

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling
x = Dense(256, activation='relu')(x)  # Add a fully connected layer
predictions = Dense(3, activation='softmax')(x)  # Output layer
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
hist=model.fit(train_generator, epochs=3, steps_per_epoch=len(train_generator))
plot_loss(hist)